# <center> <span style="color:#f6f794"> Translation from EN to ES </span> </center>
_____

### <span style="color:#f6f794"> 📚 Importing libraries and setting displays </span>

In [1]:
import pandas as pd
import requests
import gzip
from io import BytesIO
import os
import csv
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm

### <span style="color:#f6f794"> 📂 Opening files and create dataframes of them </span>

In [2]:
# Setting paths
current_dir = os.getcwd()                            # \sentiment-analyzer-inc\eda
print(current_dir)
parent_dir = os.path.dirname(current_dir)            # \sentiment-analyzer-inc
print(parent_dir)
data_dir = os.path.join(current_dir, "data")         # \sentiment-analyzer-inc\eda\data
print(data_dir)
cleaning_file_path = os.path.join(current_dir, "")   # \sentiment-analyzer-inc\eda\data\
print(cleaning_file_path)

c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda
c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc
c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda\data
c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda\


In [3]:
GITHUB_CLEAN_URL = "https://raw.githubusercontent.com/Yael-Parra/sentiment-analyzer-inc/feature/eda/eda/data/youtube_dataset_cleaned.csv.gz"


def load_airbnb_data_from_github(url):
    """
    Downloading and processing Airbnb data from GitHub repository.
    """
    print("🔗 Downloading data from GitHub...")

    try:
        # Download the compressed CSV file from GitHub
        response = requests.get(url)
        response.raise_for_status()

        print("📊 Processing compressed file..")

        # Decompressing and reading the CSV file
        with gzip.open(BytesIO(response.content), 'rt', encoding='utf-8') as f:
            df = pd.read_csv(
                f,
                sep=',',                  # Specify the delimiter
                encoding='utf-8',         # Specify the encoding
                quoting=csv.QUOTE_MINIMAL # Specify the quoting behavior
                )

        print(f"✅ Data downloaded succesfully!")
        
        return df

    except requests.exceptions.RequestException as req_err:
        print(f"❌ Network/Request Error while downloading data: {req_err}")
        return None
    except Exception as e:
        print(f"❌ General Error while processing data: {e}")
        return None

# Creating dataframe from GitHub URL
df_clean_2 = load_airbnb_data_from_github(GITHUB_CLEAN_URL)

🔗 Downloading data from GitHub...
📊 Processing compressed file..
✅ Data downloaded succesfully!


In [4]:
df_clean_2.describe()

,comment_length,word_count
count,1000.000000,1000.000000
mean,184.719000,33.776000
std,269.236313,49.069498
min,3.000000,1.000000
25%,47.000000,9.000000
50%,100.500000,19.000000
75%,216.000000,39.000000
max,4390.000000,815.000000


In [5]:
df_clean_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   comment_id           1000 non-null   object
 1   video_id             1000 non-null   object
 2   text                 1000 non-null   object
 3   is_toxic             1000 non-null   bool  
 4   is_abusive           1000 non-null   bool  
 5   is_threat            1000 non-null   bool  
 6   is_provocative       1000 non-null   bool  
 7   is_obscene           1000 non-null   bool  
 8   is_hatespeech        1000 non-null   bool  
 9   is_racist            1000 non-null   bool  
 10  is_nationalist       1000 non-null   bool  
 11  is_sexist            1000 non-null   bool  
 12  is_homophobic        1000 non-null   bool  
 13  is_religious_hate    1000 non-null   bool  
 14  is_radicalism        1000 non-null   bool  
 15  comment_length       1000 non-null   int64 
 16  word_co

traducir

In [6]:
# Load translation model
model_name = 'Helsinki-NLP/opus-mt-en-es'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate_texts(texts, batch_size=8):
    translated_texts = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Translating"):
        batch = texts[i:i+batch_size]
        tokens = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        output = model.generate(**tokens)
        translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in output]
        translated_texts.extend(translated_batch)
    return translated_texts

def augment_with_spanish(df, text_column='text'):
    df = df.copy()
    spanish_texts = translate_texts(df[text_column].tolist())
    
    df_translated = df.copy()
    df_translated[text_column] = spanish_texts

    df_augmented = pd.concat([df, df_translated], ignore_index=True)
    return df_augmented

df_augmented = augment_with_spanish(df_clean_2, text_column='text')


ImportError: 
MarianTokenizer requires the SentencePiece library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


_______
# <center> <span style="color:#f6f794"> 💾💾💾💾💾 Saving in GitHub Compressed 💾💾💾💾💾</center>

In [ ]:
# # # Saving data in cleaning_file_path and in GitHub so it can be used in the next steps and the rest of the team can access it
# cleaning_file_path = os.path.join(data_dir, "youtube_dataset_cleaned.csv.gz")

# # Saving the cleaned DataFrame to a compressed CSV file
# df_clean_1.to_csv(
#     cleaning_file_path,
#     index=False,
#     compression='gzip',
#     sep=',',                  # Crucial: Specify delimiter even for compressed
#     encoding='utf-8',         # Crucial: Specify encoding
#     quoting=csv.QUOTE_MINIMAL # Crucial: Specify quoting behavior
# )
# print(f"✅ Data saved successfully at {cleaning_file_path}")